## Install required libraries

In [ ]:
!pip install -q -U deepsparse-nightly[llm]==1.8.0.20240502 boto3==1.34.126

## Download and save the model

Here we are using a [7b Granite model](https://huggingface.co/nm-testing/granite-7b-lab-pruned50-quant-ds) that's been pruned and quantized.  
You can find more models here: [https://sparsezoo.neuralmagic.com/?modelSet=generative_ai&tasks=text_generation](https://sparsezoo.neuralmagic.com/?modelSet=generative_ai&tasks=text_generation)  
Or you can optimize your own model through [this pipeline](https://github.com/luis5tb/neural-magic-poc/blob/main/openshift-ai/pipeline_simplified.py) (requires GPU).

In [ ]:
from deepsparse import TextGeneration

model_name = "hf:nm-testing/granite-7b-lab-pruned50-quant-ds"

pipeline = TextGeneration(model=model_name)
print(pipeline.model_path)

In [ ]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)

def upload_directory_to_s3(local_directory, s3_prefix, remove_safetensors=False):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            if ".git" in relative_path:
                print(f"skipping {relative_path}")
                continue
            if remove_safetensors and ".safetensors" in relative_path:
                print(f"skipping {relative_path}")
                continue
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)


def list_objects(prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)

In [ ]:
import os

upload_directory_to_s3(os.path.dirname(pipeline.model_path), f"models/{model_name}")